# Project Statistical Analysis

## Environment

In [1]:
###### DATAFRAME ###### 
import pandas as pd
import numpy as np

###### STATISTICS ###### 
import scipy.stats as stats
from scipy.stats import t
from statsmodels.stats.proportion import proportions_ztest
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

###### VISUALIZATION ###### 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

###### SETTINGS ###### 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Initial Datasets

I will be using 2 datasets:<br>
    **- Big Mac Index** "The Economist":
    https://github.com/TheEconomist/big-mac-data<br>
    **- Economic data from countries** "Data World Bank":
    https://datacatalog.worldbank.org/dataset/world-development-indicators

### Big Mac Index (The Economist)

In [2]:
bm_data = pd.read_csv('data/big-mac-full-index.csv')

In [3]:
bm_data.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
0,2000-04-01,ARG,ARS,Argentina,2.50,1.00,2.500000,-0.004,0.050,-0.167,-0.099,1.091,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-04-01,AUS,AUD,Australia,2.59,1.68,1.541667,-0.386,-0.352,-0.486,-0.444,0.289,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-04-01,BRA,BRL,Brazil,2.95,1.79,1.648045,-0.343,-0.308,-0.451,-0.406,0.378,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,CAN,CAD,Canada,2.85,1.47,1.938776,-0.228,-0.186,-0.354,-0.301,0.622,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-04-01,CHE,CHF,Switzerland,5.90,1.70,3.470588,0.383,0.458,0.156,0.251,1.903,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
bm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1274 entries, 0 to 1273
Data columns (total 19 columns):
date             1274 non-null object
iso_a3           1274 non-null object
currency_code    1274 non-null object
name             1274 non-null object
local_price      1274 non-null float64
dollar_ex        1274 non-null float64
dollar_price     1274 non-null float64
USD_raw          1274 non-null float64
EUR_raw          1274 non-null float64
GBP_raw          1274 non-null float64
JPY_raw          1274 non-null float64
CNY_raw          1274 non-null float64
GDP_dollar       628 non-null float64
adj_price        628 non-null float64
USD_adjusted     628 non-null float64
EUR_adjusted     628 non-null float64
GBP_adjusted     628 non-null float64
JPY_adjusted     628 non-null float64
CNY_adjusted     628 non-null float64
dtypes: float64(15), object(4)
memory usage: 189.2+ KB


In [5]:
bm_data.shape

(1274, 19)

### Economic Data (World Bank)

In [6]:
#This file is more than 200Mb and to a public forked repo
#we can only upload up to 100Mb. So I've included the file 
#in the gitignore of the local repo.
#I can send you the file by mail or slack 
eco_data = pd.read_csv('data/WDI_csv/WDIData.csv')

In [7]:
eco_data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.702495,74.944860,76.258066,77.120464,78.094440,79.062000,79.851681,80.570536,81.210886,81.809630,82.368101,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN
3,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.262566,76.653498,79.021102,80.209279,84.116961,82.863365,85.083407,84.674458,86.007620,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN
4,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.558249,54.944395,56.932309,58.02529,59.727156,60.960636,59.661362,62.945506,60.944261,63.226753,64.094673,66.413865,71.817854,69.066483,72.202886,71.818284,73.466653,73.942103,75.244104,77.162305,75.538976,78.741152,79.665635,80.749293,NaN,NaN


In [8]:
eco_data.shape

(422136, 64)

In [9]:
 eco_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422136 entries, 0 to 422135
Data columns (total 64 columns):
Country Name      422136 non-null object
Country Code      422136 non-null object
Indicator Name    422136 non-null object
Indicator Code    422136 non-null object
1960              38262 non-null float64
1961              41977 non-null float64
1962              44156 non-null float64
1963              44036 non-null float64
1964              44564 non-null float64
1965              47250 non-null float64
1966              46847 non-null float64
1967              48579 non-null float64
1968              48158 non-null float64
1969              49961 non-null float64
1970              92948 non-null float64
1971              99195 non-null float64
1972              101947 non-null float64
1973              101557 non-null float64
1974              102931 non-null float64
1975              107030 non-null float64
1976              109122 non-null float64
1977              11263

### Year Selection

**2016** is the nearest year with more than 200k non null data

In [10]:
nonnulls = eco_data.shape[0] - eco_data.isnull().sum()
nonnulls.loc[nonnulls > 200000]

Country Name      422136
Country Code      422136
Indicator Name    422136
Indicator Code    422136
2000              204197
2003              200236
2004              205695
2005              219480
2006              217486
2007              221433
2008              220464
2009              221482
2010              236962
2011              227654
2012              229276
2013              224822
2014              230038
2015              224078
2016              215692
dtype: int64

## Cleaning Datasets

### Big Mac index 2016 Selection

In [11]:
bm_2016 = bm_data.loc[bm_data['date'].str.contains('2016')]

In [12]:
bm_2016.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
891,2016-01-01,ARE,AED,UAE,13.00,3.673100,3.539245,-0.282,-0.115,-0.162,0.135,0.319,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892,2016-01-01,ARG,ARS,Argentina,33.00,13.809250,2.389703,-0.515,-0.402,-0.434,-0.234,-0.110,12735.196,2.744474,-0.247,-0.186,-0.185,0.006,-0.171
893,2016-01-01,AUS,AUD,Australia,5.30,1.415729,3.743655,-0.241,-0.064,-0.113,0.200,0.395,61066.154,4.509511,-0.282,-0.224,-0.223,-0.041,-0.209
894,2016-01-01,BRA,BRL,Brazil,13.50,4.024800,3.354204,-0.320,-0.161,-0.206,0.076,0.250,11572.700,2.702020,0.074,0.161,0.161,0.434,0.182
895,2016-01-01,CAN,CAD,Canada,5.84,1.409200,4.144195,-0.159,0.036,-0.018,0.329,0.544,50304.025,4.116480,-0.129,-0.058,-0.058,0.163,-0.041


In [13]:
bm_2016 = bm_2016[['date','iso_a3','currency_code','name','local_price','dollar_ex','dollar_price']]

In [14]:
bm_2016.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
891,2016-01-01,ARE,AED,UAE,13.00,3.673100,3.539245
892,2016-01-01,ARG,ARS,Argentina,33.00,13.809250,2.389703
893,2016-01-01,AUS,AUD,Australia,5.30,1.415729,3.743655
894,2016-01-01,BRA,BRL,Brazil,13.50,4.024800,3.354204
895,2016-01-01,CAN,CAD,Canada,5.84,1.409200,4.144195


In [15]:
len(bm_2016.iso_a3.unique())

43

In [16]:
bm_2016.shape

(86, 7)

In [17]:
bm_2016.date.unique()

array(['2016-01-01', '2016-07-01'], dtype=object)

We have the Big Mac Index data of 2016 for 43 countries, 2 rows per country. We will select only oine value: 1/1/2016

In [18]:
bm_2016 = bm_2016.loc[bm_2016['date'] == '2016-01-01'].sort_values(by='iso_a3').reset_index()

In [19]:
bm_2016 = bm_2016.drop(columns='index')

In [20]:
bm_2016.shape

(43, 7)

In [21]:
bm_2016.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
0,2016-01-01,ARE,AED,UAE,13.00,3.673100,3.539245
1,2016-01-01,ARG,ARS,Argentina,33.00,13.809250,2.389703
2,2016-01-01,AUS,AUD,Australia,5.30,1.415729,3.743655
3,2016-01-01,BRA,BRL,Brazil,13.50,4.024800,3.354204
4,2016-01-01,CAN,CAD,Canada,5.84,1.409200,4.144195


In [22]:
bm_2016.tail()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
38,2016-01-01,UKR,UAH,Ukraine,36.00,23.35000,1.541756
39,2016-01-01,URY,UYU,Uruguay,113.00,30.18500,3.743581
40,2016-01-01,USA,USD,United States,4.93,1.00000,4.930000
41,2016-01-01,VNM,VND,Vietnam,60000.00,22467.50000,2.670524
42,2016-01-01,ZAF,ZAR,South Africa,28.00,15.81375,1.770611


In [23]:
countries = list(bm_2016.iso_a3.unique())

In [24]:
countries

['ARE',
 'ARG',
 'AUS',
 'BRA',
 'CAN',
 'CHE',
 'CHL',
 'CHN',
 'COL',
 'CRI',
 'CZE',
 'DNK',
 'EGY',
 'EUZ',
 'GBR',
 'HKG',
 'HUN',
 'IDN',
 'IND',
 'ISR',
 'JPN',
 'KOR',
 'LKA',
 'MEX',
 'MYS',
 'NOR',
 'NZL',
 'PAK',
 'PER',
 'PHL',
 'POL',
 'RUS',
 'SAU',
 'SGP',
 'SWE',
 'THA',
 'TUR',
 'TWN',
 'UKR',
 'URY',
 'USA',
 'VNM',
 'ZAF']

**note:** the Euro Area has a unique price for the Big Mac

### Economic data 2016 Selection

I will make a pivot table with the 2016 indicators as columns, the 2016 values as cells and a column with the country code.

In [25]:
eco_data.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', 'Unnamed: 63'], dtype='object')

In [26]:
eco_2016 = eco_data[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code','2016']]

In [27]:
eco_2016.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2016
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,84.510171
3,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,89.678685
4,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,79.665635


In [28]:
eco_2016.shape

(422136, 5)

In [29]:
eco_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422136 entries, 0 to 422135
Data columns (total 5 columns):
Country Name      422136 non-null object
Country Code      422136 non-null object
Indicator Name    422136 non-null object
Indicator Code    422136 non-null object
2016              215692 non-null float64
dtypes: float64(1), object(4)
memory usage: 16.1+ MB


In [30]:
eco_2016_pivot = pd.pivot_table(eco_2016, values=['2016'], 
index=['Country Code'], columns='Indicator Code',).reset_index()

In [31]:
eco_2016_pivot.shape

(263, 1419)

In [32]:
eco_2016_pivot.head()

Country Code              2016                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       \
Indicator Code              AG.CON.FERT.PT.ZS AG.CON.FERT.ZS AG.LND.AGRI.K2 AG.LND.AGRI.ZS AG.LND.ARBL.HA AG.LND.ARBL.HA.PC AG.LND.ARBL.ZS AG.LND.CREL.HA AG.LND.CROP.ZS AG.LND.FRST.K2 AG.LND.FRST.ZS AG.LND.IRIG.AG.ZS AG.LND.TOTL.K2 AG.PRD.CREL.MT AG.PRD.CROP.XD AG.PRD.FOOD.XD AG.PRD.LVSK.XD AG.SRF.TOTL.K2 AG.YLD.CREL.KG BG.GSR.NFSV.GD.ZS BM.GSR.CMCP.ZS BM.GSR.FCTY.CD BM.GSR.GNFS.CD BM.GSR.INSF.ZS BM.GSR.MRCH.CD BM.GSR.NFSV.CD BM.GSR.ROYL.CD BM.GSR.TOTL.CD BM.GSR.TRAN.ZS BM.GSR.TRVL.ZS BM.KLT.DINV.CD.WD BM.KLT.DINV.WD.GD.ZS BM.TRF.PRVT.CD BM.TRF.PWKR.CD.DT BN.CAB.XOKA.CD BN.CAB.XOKA.GD.ZS BN.FIN.TOTL.CD BN.GSR.FCTY.CD BN.GSR.GNFS.CD BN.GSR.MRCH.CD BN.KAC.EOMS.CD BN.KLT.DINV.CD BN.KLT.PTXL.CD BN.RES.INCL.CD BN.TRF.CURR.CD BN.TRF.KOGT.CD BX.GRT.EXTA.CD.WD BX.GRT.TECH.CD.WD BX.GSR.CCIS.CD BX.GSR.CCIS.ZS BX.GSR.CMCP.ZS BX.GSR.FCTY.CD BX.GSR.GNFS.CD BX.GSR.INSF.ZS BX.GSR.MRCH.CD BX.GSR.NFSV.CD BX.GSR.ROYL.CD BX.GSR.TOTL.CD BX.GSR.TRAN.ZS BX.GSR.TRVL.ZS BX.KLT.DINV.CD.WD   
0                       ABW               NaN            NaN      20.000000      11.111111   2.000000e+03          0.019071      11.111111            NaN            NaN       4.200000       2.333333               NaN          180.0            NaN            NaN            NaN            NaN          180.0            NaN        110.692401      46.932941   1.869838e+08   2.008884e+09       3.665942   1.141899e+09   8.669852e+08   1.632750e+07   2.195868e+09      12.671845      36.746703      4.936101e+05             0.018648   1.389522e+08      7.174353e+07   1.410695e+08          5.329558   1.330472e+08  -1.348016e+08   3.371860e+08  -8.587910e+08  -1.592853e+07  -2.048856e+07  -3.900896e+07   9.703911e+07  -6.131491e+07   7.605834e+06               NaN               NaN   2.211410e+07       1.071958      14.898422   5.218216e+07   2.346070e+09       0.816978   2.831080e+08   2.062962e+09   2.786592e+04   2.398252e+09       5.230960      79.053439      2.098217e+07   
1                       AFG        590.067055      12.182300  379100.000000      58.067580   7.729000e+06          0.218437      11.838679      2793133.0       0.277242   13500.000000       2.067825          6.481140       652860.0      5534902.0         147.96         125.41          98.70       652860.0         1981.6          7.448810      13.162094   7.070300e+07   7.469468e+09      12.229359   6.536110e+09   9.333575e+08   3.680295e+04   7.540171e+09      66.134391       8.474156     -1.477594e+07            -0.076312   1.197707e+08      1.829258e+08  -3.322760e+09        -17.160673   6.901914e+08   9.457179e+07  -6.346325e+09  -5.921896e+09   2.241923e+09  -1.083673e+08   9.869677e+07   7.136550e+08   2.928993e+09   1.771028e+09      3.642310e+09       490400000.0   7.792645e+07      15.311857      78.638417   1.652748e+08   1.123143e+09       4.743911   6.142139e+08   5.089288e+08            NaN   1.288417e+09       7.037476       9.580196      9.359132e+07   
2                       AGO               NaN       7.982743  591900.000000      47.477

In [33]:
eco_2016_stack = eco_2016_pivot.iloc[:,1:].stack(level=0)

In [34]:
countries_eco = list(eco_2016_pivot['Country Code'])

In [35]:
eco_2016_stack['Country Code'] = countries_eco

In [36]:
eco_2016_stack.head()

,Indicator Code,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,AG.LND.CROP.ZS,AG.LND.FRST.K2,AG.LND.FRST.ZS,AG.LND.IRIG.AG.ZS,AG.LND.TOTL.K2,AG.PRD.CREL.MT,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,AG.YLD.CREL.KG,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT,BN.CAB.XOKA.CD,BN.CAB.XOKA.GD.ZS,BN.FIN.TOTL.CD,BN.GSR.FCTY.CD,BN.GSR.GNFS.CD,BN.GSR.MRCH.CD,BN.KAC.EOMS.CD,BN.KLT.DINV.CD,BN.KLT.PTXL.CD,BN.RES.INCL.CD,BN.TRF.CURR.CD,BN.TRF.KOGT.CD,BX.GRT.EXTA.CD.WD,BX.GRT.TECH.CD.WD,BX.GSR.CCIS.CD,BX.GSR.CCIS.ZS,BX.GSR.CMCP.ZS,BX.GSR.FCTY.CD,BX.GSR.GNFS.CD,BX.GSR.INSF.ZS,BX.GSR.MRCH.CD,BX.GSR.NFSV.CD,BX.GSR.ROYL.CD,BX.GSR.TOTL.CD,BX.GSR.TRAN.ZS,BX.GSR.TRVL.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,BX.KLT.DREM.CD.DT,BX.PEF.TOTL.CD.WD,BX.TRF.CURR.CD,BX.TRF.PWKR.CD,BX.TRF.PWKR.CD.DT,BX.TRF.PWKR.DT.GD.ZS,CM.MKT.INDX.ZG,CM.MKT.LCAP.CD,CM.MKT.LCAP.GD.ZS,CM.MKT.LDOM.NO,CM.MKT.TRAD.CD,CM.MKT.TRAD.GD.ZS,CM.MKT.TRNR,DC.DAC.AUSL.CD,DC.DAC.AUTL.CD,DC.DAC.BELL.CD,DC.DAC.CANL.CD,DC.DAC.CECL.CD,DC.DAC.CHEL.CD,DC.DAC.CZEL.CD,DC.DAC.DEUL.CD,DC.DAC.DNKL.CD,DC.DAC.ESPL.CD,DC.DAC.FINL.CD,DC.DAC.FRAL.CD,DC.DAC.GBRL.CD,DC.DAC.GRCL.CD,DC.DAC.IRLL.CD,DC.DAC.ISLL.CD,DC.DAC.ITAL.CD,DC.DAC.JPNL.CD,DC.DAC.KORL.CD,DC.DAC.LUXL.CD,DC.DAC.NLDL.CD,DC.DAC.NORL.CD,DC.DAC.NZLL.CD,DC.DAC.POLL.CD,DC.DAC.PRTL.CD,DC.DAC.SVKL.CD,DC.DAC.SVNL.CD,DC.DAC.SWEL.CD,DC.DAC.TOTL.CD,DC.DAC.USAL.CD,DC.ODA.TLDC.CD,DC.ODA.TLDC.GN.ZS,DC.ODA.TOTL.CD,DC.ODA.TOTL.GN.ZS,DC.ODA.TOTL.KD,DT.AMT.BLAT.CD,DT.AMT.BLTC.CD,DT.AMT.DIMF.CD,DT.AMT.DLTF.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.DPPG.CD,DT.AMT.MIBR.CD,DT.AMT.MIDA.CD,DT.AMT.MLAT.CD,DT.AMT.MLTC.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PNGB.CD,DT.AMT.PNGC.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.AXA.DPPG.CD,DT.AXA.OFFT.CD,DT.AXA.PRVT.CD,DT.AXF.DPPG.CD,DT.AXR.DPPG.CD,DT.AXR.OFFT.CD,DT.AXR.PRVT.CD,DT.COM.DPPG.CD,DT.COM.MIBR.CD,DT.COM.MIDA.CD,DT.COM.OFFT.CD,DT.COM.PRVT.CD,DT.CUR.DMAK.ZS,DT.CUR.EURO.ZS,DT.CUR.FFRC.ZS,DT.CUR.JYEN.ZS,DT.CUR.MULC.ZS,DT.CUR.OTHC.ZS,DT.CUR.SDRW.ZS,DT.CUR.SWFR.ZS,DT.CUR.UKPS.ZS,DT.CUR.USDL.ZS,DT.DFR.DPPG.CD,DT.DIS.BLAT.CD,DT.DIS.BLTC.CD,DT.DIS.DIMF.CD,DT.DIS.DLTF.CD,DT.DIS.DLXF.CD,DT.DIS.DPNG.CD,DT.DIS.DPPG.CD,DT.DIS.IDAG.CD,DT.DIS.MIBR.CD,DT.DIS.MIDA.CD,DT.DIS.MLAT.CD,DT.DIS.MLTC.CD,DT.DIS.OFFT.CD,DT.DIS.PBND.CD,DT.DIS.PCBK.CD,DT.DIS.PNGB.CD,DT.DIS.PNGC.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.DOD.ALLC.CD,DT.DOD.ALLC.ZS,DT.DOD.BLAT.CD,DT.DOD.BLTC.CD,DT.DOD.DECT.CD,DT.DOD.DECT.CD.CG,DT.DOD.DECT.EX.ZS,DT.DOD.DECT.GN.ZS,DT.DOD.DIMF.CD,DT.DOD.DLXF.CD,DT.DOD.DPNG.CD,DT.DOD.DPPG.CD,DT.DOD.DSTC.CD,DT.DOD.DSTC.IR.ZS,DT.DOD.DSTC.XP.ZS,DT.DOD.DSTC.ZS,DT.DOD.MDRI.CD,DT.DOD.MIBR.CD,DT.DOD.MIDA.CD,DT.DOD.MLAT.CD,DT.DOD.MLAT.ZS,DT.DOD.MLTC.CD,DT.DOD.MWBG.CD,DT.DOD.OFFT.CD,DT.DOD.PBND.CD,DT.DOD.PCBK.CD,DT.DOD.PNGB.CD,DT.DOD.PNGC.CD,DT.DOD.PROP.CD,DT.DOD.PRVS.CD,DT.DOD.PRVT.CD,DT.DOD.PUBS.CD,DT.DOD.PVLX.CD,DT.DOD.PVLX.EX.ZS,DT.DOD.PVLX.GN.ZS,DT.DOD.RSDL.CD,DT.DOD.VTOT.CD,DT.DSB.DPPG.CD,DT.DSF.DPPG.CD,DT.DXR.DPPG.CD,DT.GPA.DPPG,DT.GPA.OFFT,DT.GPA.PRVT,DT.GRE.DPPG,DT.GRE.OFFT,DT.GRE.PRVT,DT.INR.DPPG,DT.INR.OFFT,DT.INR.PRVT,DT.INT.BLAT.CD,DT.INT.BLTC.CD,DT.INT.DECT.CD,DT.INT.DECT.EX.ZS,DT.INT.DECT.GN.ZS,DT.INT.DIMF.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.DPPG.CD,DT.INT.DSTC.CD,DT.INT.MIBR.CD,DT.INT.MIDA.CD,DT.INT.MLAT.CD,DT.INT.MLTC.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PNGB.CD,DT.INT.PNGC.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD,DT.IXA.DPPG.CD,DT.IXA.DPPG.CD.CG,DT.IXA.OFFT.CD,DT.IXA.PRVT.CD,DT.IXF.DPPG.CD,DT.IXR.DPPG.CD,DT.IXR.OFFT.CD,DT.IXR.PRVT.CD,DT.MAT.DPPG,DT.MAT.OFFT,...,SL.UEM.1524.NE.ZS,SL.UEM.1524.ZS,SL.UEM.ADVN.FE.ZS,SL.UEM.ADVN.MA.ZS,SL.UEM.ADVN.ZS,SL.UEM.BASC.FE.ZS,SL.UEM.BASC.MA.ZS,SL.UEM.BASC.ZS,SL.UEM.INTM.FE.ZS,SL.UEM.INTM.MA

In [37]:
cols = list(eco_2016_stack.columns)
cols = [cols[-1]] + cols[:-1]
eco_2016_stack = eco_2016_stack[cols]
eco_2016_stack.head()

,Indicator Code,Country Code,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,AG.LND.CROP.ZS,AG.LND.FRST.K2,AG.LND.FRST.ZS,AG.LND.IRIG.AG.ZS,AG.LND.TOTL.K2,AG.PRD.CREL.MT,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,AG.YLD.CREL.KG,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT,BN.CAB.XOKA.CD,BN.CAB.XOKA.GD.ZS,BN.FIN.TOTL.CD,BN.GSR.FCTY.CD,BN.GSR.GNFS.CD,BN.GSR.MRCH.CD,BN.KAC.EOMS.CD,BN.KLT.DINV.CD,BN.KLT.PTXL.CD,BN.RES.INCL.CD,BN.TRF.CURR.CD,BN.TRF.KOGT.CD,BX.GRT.EXTA.CD.WD,BX.GRT.TECH.CD.WD,BX.GSR.CCIS.CD,BX.GSR.CCIS.ZS,BX.GSR.CMCP.ZS,BX.GSR.FCTY.CD,BX.GSR.GNFS.CD,BX.GSR.INSF.ZS,BX.GSR.MRCH.CD,BX.GSR.NFSV.CD,BX.GSR.ROYL.CD,BX.GSR.TOTL.CD,BX.GSR.TRAN.ZS,BX.GSR.TRVL.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,BX.KLT.DREM.CD.DT,BX.PEF.TOTL.CD.WD,BX.TRF.CURR.CD,BX.TRF.PWKR.CD,BX.TRF.PWKR.CD.DT,BX.TRF.PWKR.DT.GD.ZS,CM.MKT.INDX.ZG,CM.MKT.LCAP.CD,CM.MKT.LCAP.GD.ZS,CM.MKT.LDOM.NO,CM.MKT.TRAD.CD,CM.MKT.TRAD.GD.ZS,CM.MKT.TRNR,DC.DAC.AUSL.CD,DC.DAC.AUTL.CD,DC.DAC.BELL.CD,DC.DAC.CANL.CD,DC.DAC.CECL.CD,DC.DAC.CHEL.CD,DC.DAC.CZEL.CD,DC.DAC.DEUL.CD,DC.DAC.DNKL.CD,DC.DAC.ESPL.CD,DC.DAC.FINL.CD,DC.DAC.FRAL.CD,DC.DAC.GBRL.CD,DC.DAC.GRCL.CD,DC.DAC.IRLL.CD,DC.DAC.ISLL.CD,DC.DAC.ITAL.CD,DC.DAC.JPNL.CD,DC.DAC.KORL.CD,DC.DAC.LUXL.CD,DC.DAC.NLDL.CD,DC.DAC.NORL.CD,DC.DAC.NZLL.CD,DC.DAC.POLL.CD,DC.DAC.PRTL.CD,DC.DAC.SVKL.CD,DC.DAC.SVNL.CD,DC.DAC.SWEL.CD,DC.DAC.TOTL.CD,DC.DAC.USAL.CD,DC.ODA.TLDC.CD,DC.ODA.TLDC.GN.ZS,DC.ODA.TOTL.CD,DC.ODA.TOTL.GN.ZS,DC.ODA.TOTL.KD,DT.AMT.BLAT.CD,DT.AMT.BLTC.CD,DT.AMT.DIMF.CD,DT.AMT.DLTF.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.DPPG.CD,DT.AMT.MIBR.CD,DT.AMT.MIDA.CD,DT.AMT.MLAT.CD,DT.AMT.MLTC.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PNGB.CD,DT.AMT.PNGC.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.AXA.DPPG.CD,DT.AXA.OFFT.CD,DT.AXA.PRVT.CD,DT.AXF.DPPG.CD,DT.AXR.DPPG.CD,DT.AXR.OFFT.CD,DT.AXR.PRVT.CD,DT.COM.DPPG.CD,DT.COM.MIBR.CD,DT.COM.MIDA.CD,DT.COM.OFFT.CD,DT.COM.PRVT.CD,DT.CUR.DMAK.ZS,DT.CUR.EURO.ZS,DT.CUR.FFRC.ZS,DT.CUR.JYEN.ZS,DT.CUR.MULC.ZS,DT.CUR.OTHC.ZS,DT.CUR.SDRW.ZS,DT.CUR.SWFR.ZS,DT.CUR.UKPS.ZS,DT.CUR.USDL.ZS,DT.DFR.DPPG.CD,DT.DIS.BLAT.CD,DT.DIS.BLTC.CD,DT.DIS.DIMF.CD,DT.DIS.DLTF.CD,DT.DIS.DLXF.CD,DT.DIS.DPNG.CD,DT.DIS.DPPG.CD,DT.DIS.IDAG.CD,DT.DIS.MIBR.CD,DT.DIS.MIDA.CD,DT.DIS.MLAT.CD,DT.DIS.MLTC.CD,DT.DIS.OFFT.CD,DT.DIS.PBND.CD,DT.DIS.PCBK.CD,DT.DIS.PNGB.CD,DT.DIS.PNGC.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.DOD.ALLC.CD,DT.DOD.ALLC.ZS,DT.DOD.BLAT.CD,DT.DOD.BLTC.CD,DT.DOD.DECT.CD,DT.DOD.DECT.CD.CG,DT.DOD.DECT.EX.ZS,DT.DOD.DECT.GN.ZS,DT.DOD.DIMF.CD,DT.DOD.DLXF.CD,DT.DOD.DPNG.CD,DT.DOD.DPPG.CD,DT.DOD.DSTC.CD,DT.DOD.DSTC.IR.ZS,DT.DOD.DSTC.XP.ZS,DT.DOD.DSTC.ZS,DT.DOD.MDRI.CD,DT.DOD.MIBR.CD,DT.DOD.MIDA.CD,DT.DOD.MLAT.CD,DT.DOD.MLAT.ZS,DT.DOD.MLTC.CD,DT.DOD.MWBG.CD,DT.DOD.OFFT.CD,DT.DOD.PBND.CD,DT.DOD.PCBK.CD,DT.DOD.PNGB.CD,DT.DOD.PNGC.CD,DT.DOD.PROP.CD,DT.DOD.PRVS.CD,DT.DOD.PRVT.CD,DT.DOD.PUBS.CD,DT.DOD.PVLX.CD,DT.DOD.PVLX.EX.ZS,DT.DOD.PVLX.GN.ZS,DT.DOD.RSDL.CD,DT.DOD.VTOT.CD,DT.DSB.DPPG.CD,DT.DSF.DPPG.CD,DT.DXR.DPPG.CD,DT.GPA.DPPG,DT.GPA.OFFT,DT.GPA.PRVT,DT.GRE.DPPG,DT.GRE.OFFT,DT.GRE.PRVT,DT.INR.DPPG,DT.INR.OFFT,DT.INR.PRVT,DT.INT.BLAT.CD,DT.INT.BLTC.CD,DT.INT.DECT.CD,DT.INT.DECT.EX.ZS,DT.INT.DECT.GN.ZS,DT.INT.DIMF.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.DPPG.CD,DT.INT.DSTC.CD,DT.INT.MIBR.CD,DT.INT.MIDA.CD,DT.INT.MLAT.CD,DT.INT.MLTC.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PNGB.CD,DT.INT.PNGC.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD,DT.IXA.DPPG.CD,DT.IXA.DPPG.CD.CG,DT.IXA.OFFT.CD,DT.IXA.PRVT.CD,DT.IXF.DPPG.CD,DT.IXR.DPPG.CD,DT.IXR.OFFT.CD,DT.IXR.PRVT.CD,DT.MAT.DPPG,...,SL.UEM.1524.MA.ZS,SL.UEM.1524.NE.ZS,SL.UEM.1524.ZS,SL.UEM.ADVN.FE.ZS,SL.UEM.ADVN.MA.ZS,SL.UEM.ADVN.ZS,SL.UEM.BASC.FE.ZS,SL.UEM.BASC.MA.ZS,SL.UEM.BASC.ZS,SL.UEM.INTM.F

In [38]:
eco_2016_stack.index = eco_2016_stack.index.droplevel(level=1)

In [39]:
eco_2016_stack.head()

Indicator Code,Country Code,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,AG.LND.CROP.ZS,AG.LND.FRST.K2,AG.LND.FRST.ZS,AG.LND.IRIG.AG.ZS,AG.LND.TOTL.K2,AG.PRD.CREL.MT,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,AG.YLD.CREL.KG,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT,BN.CAB.XOKA.CD,BN.CAB.XOKA.GD.ZS,BN.FIN.TOTL.CD,BN.GSR.FCTY.CD,BN.GSR.GNFS.CD,BN.GSR.MRCH.CD,BN.KAC.EOMS.CD,BN.KLT.DINV.CD,BN.KLT.PTXL.CD,BN.RES.INCL.CD,BN.TRF.CURR.CD,BN.TRF.KOGT.CD,BX.GRT.EXTA.CD.WD,BX.GRT.TECH.CD.WD,BX.GSR.CCIS.CD,BX.GSR.CCIS.ZS,BX.GSR.CMCP.ZS,BX.GSR.FCTY.CD,BX.GSR.GNFS.CD,BX.GSR.INSF.ZS,BX.GSR.MRCH.CD,BX.GSR.NFSV.CD,BX.GSR.ROYL.CD,BX.GSR.TOTL.CD,BX.GSR.TRAN.ZS,BX.GSR.TRVL.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,BX.KLT.DREM.CD.DT,BX.PEF.TOTL.CD.WD,BX.TRF.CURR.CD,BX.TRF.PWKR.CD,BX.TRF.PWKR.CD.DT,BX.TRF.PWKR.DT.GD.ZS,CM.MKT.INDX.ZG,CM.MKT.LCAP.CD,CM.MKT.LCAP.GD.ZS,CM.MKT.LDOM.NO,CM.MKT.TRAD.CD,CM.MKT.TRAD.GD.ZS,CM.MKT.TRNR,DC.DAC.AUSL.CD,DC.DAC.AUTL.CD,DC.DAC.BELL.CD,DC.DAC.CANL.CD,DC.DAC.CECL.CD,DC.DAC.CHEL.CD,DC.DAC.CZEL.CD,DC.DAC.DEUL.CD,DC.DAC.DNKL.CD,DC.DAC.ESPL.CD,DC.DAC.FINL.CD,DC.DAC.FRAL.CD,DC.DAC.GBRL.CD,DC.DAC.GRCL.CD,DC.DAC.IRLL.CD,DC.DAC.ISLL.CD,DC.DAC.ITAL.CD,DC.DAC.JPNL.CD,DC.DAC.KORL.CD,DC.DAC.LUXL.CD,DC.DAC.NLDL.CD,DC.DAC.NORL.CD,DC.DAC.NZLL.CD,DC.DAC.POLL.CD,DC.DAC.PRTL.CD,DC.DAC.SVKL.CD,DC.DAC.SVNL.CD,DC.DAC.SWEL.CD,DC.DAC.TOTL.CD,DC.DAC.USAL.CD,DC.ODA.TLDC.CD,DC.ODA.TLDC.GN.ZS,DC.ODA.TOTL.CD,DC.ODA.TOTL.GN.ZS,DC.ODA.TOTL.KD,DT.AMT.BLAT.CD,DT.AMT.BLTC.CD,DT.AMT.DIMF.CD,DT.AMT.DLTF.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.DPPG.CD,DT.AMT.MIBR.CD,DT.AMT.MIDA.CD,DT.AMT.MLAT.CD,DT.AMT.MLTC.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PNGB.CD,DT.AMT.PNGC.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.AXA.DPPG.CD,DT.AXA.OFFT.CD,DT.AXA.PRVT.CD,DT.AXF.DPPG.CD,DT.AXR.DPPG.CD,DT.AXR.OFFT.CD,DT.AXR.PRVT.CD,DT.COM.DPPG.CD,DT.COM.MIBR.CD,DT.COM.MIDA.CD,DT.COM.OFFT.CD,DT.COM.PRVT.CD,DT.CUR.DMAK.ZS,DT.CUR.EURO.ZS,DT.CUR.FFRC.ZS,DT.CUR.JYEN.ZS,DT.CUR.MULC.ZS,DT.CUR.OTHC.ZS,DT.CUR.SDRW.ZS,DT.CUR.SWFR.ZS,DT.CUR.UKPS.ZS,DT.CUR.USDL.ZS,DT.DFR.DPPG.CD,DT.DIS.BLAT.CD,DT.DIS.BLTC.CD,DT.DIS.DIMF.CD,DT.DIS.DLTF.CD,DT.DIS.DLXF.CD,DT.DIS.DPNG.CD,DT.DIS.DPPG.CD,DT.DIS.IDAG.CD,DT.DIS.MIBR.CD,DT.DIS.MIDA.CD,DT.DIS.MLAT.CD,DT.DIS.MLTC.CD,DT.DIS.OFFT.CD,DT.DIS.PBND.CD,DT.DIS.PCBK.CD,DT.DIS.PNGB.CD,DT.DIS.PNGC.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.DOD.ALLC.CD,DT.DOD.ALLC.ZS,DT.DOD.BLAT.CD,DT.DOD.BLTC.CD,DT.DOD.DECT.CD,DT.DOD.DECT.CD.CG,DT.DOD.DECT.EX.ZS,DT.DOD.DECT.GN.ZS,DT.DOD.DIMF.CD,DT.DOD.DLXF.CD,DT.DOD.DPNG.CD,DT.DOD.DPPG.CD,DT.DOD.DSTC.CD,DT.DOD.DSTC.IR.ZS,DT.DOD.DSTC.XP.ZS,DT.DOD.DSTC.ZS,DT.DOD.MDRI.CD,DT.DOD.MIBR.CD,DT.DOD.MIDA.CD,DT.DOD.MLAT.CD,DT.DOD.MLAT.ZS,DT.DOD.MLTC.CD,DT.DOD.MWBG.CD,DT.DOD.OFFT.CD,DT.DOD.PBND.CD,DT.DOD.PCBK.CD,DT.DOD.PNGB.CD,DT.DOD.PNGC.CD,DT.DOD.PROP.CD,DT.DOD.PRVS.CD,DT.DOD.PRVT.CD,DT.DOD.PUBS.CD,DT.DOD.PVLX.CD,DT.DOD.PVLX.EX.ZS,DT.DOD.PVLX.GN.ZS,DT.DOD.RSDL.CD,DT.DOD.VTOT.CD,DT.DSB.DPPG.CD,DT.DSF.DPPG.CD,DT.DXR.DPPG.CD,DT.GPA.DPPG,DT.GPA.OFFT,DT.GPA.PRVT,DT.GRE.DPPG,DT.GRE.OFFT,DT.GRE.PRVT,DT.INR.DPPG,DT.INR.OFFT,DT.INR.PRVT,DT.INT.BLAT.CD,DT.INT.BLTC.CD,DT.INT.DECT.CD,DT.INT.DECT.EX.ZS,DT.INT.DECT.GN.ZS,DT.INT.DIMF.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.DPPG.CD,DT.INT.DSTC.CD,DT.INT.MIBR.CD,DT.INT.MIDA.CD,DT.INT.MLAT.CD,DT.INT.MLTC.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PNGB.CD,DT.INT.PNGC.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD,DT.IXA.DPPG.CD,DT.IXA.DPPG.CD.CG,DT.IXA.OFFT.CD,DT.IXA.PRVT.CD,DT.IXF.DPPG.CD,DT.IXR.DPPG.CD,DT.IXR.OFFT.CD,DT.IXR.PRVT.CD,DT.MAT.DPPG,...,SL.UEM.1524.MA.ZS,SL.UEM.1524.NE.ZS,SL.UEM.1524.ZS,SL.UEM.ADVN.FE.ZS,SL.UEM.ADVN.MA.ZS,SL.UEM.ADVN.ZS,SL.UEM.BASC.FE.ZS,SL.UEM.BASC.MA.ZS,SL.UEM.BASC.ZS,SL.UEM.INTM.FE

As I have lost the deffinition of each indicator, I'll create a indicators definitions dataset

In [40]:
indicators = eco_2016[['Indicator Code','Indicator Name']].sort_values(by='Indicator Code')
indicators = indicators.drop_duplicates('Indicator Code', keep='first').reset_index()
indicators = indicators.drop('index', axis=1)

In [41]:
indicators.head()

,Indicator Code,Indicator Name
0,AG.AGR.TRAC.NO,"Agricultural machinery, tractors"
1,AG.CON.FERT.PT.ZS,Fertilizer consumption (% of fertilizer produc...
2,AG.CON.FERT.ZS,Fertilizer consumption (kilograms per hectare ...
3,AG.LND.AGRI.K2,Agricultural land (sq. km)
4,AG.LND.AGRI.ZS,Agricultural land (% of land area)


The indicators Dataset explain the definition of the columns

In [42]:
indicators.shape

(1599, 2)

### Economic data for the selected countries

As we have the EuroZone with a unique price for the index, and Germany had the bigger nominal GDP in the European union I will use Germanies data for the project as Eurozone.

![European nominal GDP 2016](img/GDP-2016.png)

In [43]:
countries.remove('EUZ')

In [44]:
countries.append('DEU')

In [45]:
countries.sort()

In [46]:
len(countries)

43

Add Germany with the EUZ data in the Big Mac Dataframe

In [47]:
bm_2016.loc[43] = ['2016-01-01', 'DEU', 'EUR', 'Germany','3.72','0.930233','3.999']

In [48]:
bm_2016.loc[bm_2016['iso_a3'] == 'DEU']

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
43,2016-01-01,DEU,EUR,Germany,3.72,0.930233,3.999


In [49]:
eco_2016_sel = eco_2016_stack[eco_2016_stack['Country Code'].isin(countries)].reset_index()

In [50]:
eco_2016_sel = eco_2016_sel.drop('index', axis=1)

In [51]:
eco_2016_sel.shape

(42, 1419)

One of the countries from the index do not have any data. 

In [52]:
countries2 = eco_2016_sel['Country Code']

In [53]:
set(countries) - set(countries2)

{'TWN'}

The country is Taiwan. We will keep the economic info of the 42 countries shared in the 2 datasets.

In [54]:
eco_2016_sel.head()

Indicator Code,Country Code,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CREL.HA,AG.LND.CROP.ZS,AG.LND.FRST.K2,AG.LND.FRST.ZS,AG.LND.IRIG.AG.ZS,AG.LND.TOTL.K2,AG.PRD.CREL.MT,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,AG.YLD.CREL.KG,BG.GSR.NFSV.GD.ZS,BM.GSR.CMCP.ZS,BM.GSR.FCTY.CD,BM.GSR.GNFS.CD,BM.GSR.INSF.ZS,BM.GSR.MRCH.CD,BM.GSR.NFSV.CD,BM.GSR.ROYL.CD,BM.GSR.TOTL.CD,BM.GSR.TRAN.ZS,BM.GSR.TRVL.ZS,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BM.TRF.PRVT.CD,BM.TRF.PWKR.CD.DT,BN.CAB.XOKA.CD,BN.CAB.XOKA.GD.ZS,BN.FIN.TOTL.CD,BN.GSR.FCTY.CD,BN.GSR.GNFS.CD,BN.GSR.MRCH.CD,BN.KAC.EOMS.CD,BN.KLT.DINV.CD,BN.KLT.PTXL.CD,BN.RES.INCL.CD,BN.TRF.CURR.CD,BN.TRF.KOGT.CD,BX.GRT.EXTA.CD.WD,BX.GRT.TECH.CD.WD,BX.GSR.CCIS.CD,BX.GSR.CCIS.ZS,BX.GSR.CMCP.ZS,BX.GSR.FCTY.CD,BX.GSR.GNFS.CD,BX.GSR.INSF.ZS,BX.GSR.MRCH.CD,BX.GSR.NFSV.CD,BX.GSR.ROYL.CD,BX.GSR.TOTL.CD,BX.GSR.TRAN.ZS,BX.GSR.TRVL.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,BX.KLT.DREM.CD.DT,BX.PEF.TOTL.CD.WD,BX.TRF.CURR.CD,BX.TRF.PWKR.CD,BX.TRF.PWKR.CD.DT,BX.TRF.PWKR.DT.GD.ZS,CM.MKT.INDX.ZG,CM.MKT.LCAP.CD,CM.MKT.LCAP.GD.ZS,CM.MKT.LDOM.NO,CM.MKT.TRAD.CD,CM.MKT.TRAD.GD.ZS,CM.MKT.TRNR,DC.DAC.AUSL.CD,DC.DAC.AUTL.CD,DC.DAC.BELL.CD,DC.DAC.CANL.CD,DC.DAC.CECL.CD,DC.DAC.CHEL.CD,DC.DAC.CZEL.CD,DC.DAC.DEUL.CD,DC.DAC.DNKL.CD,DC.DAC.ESPL.CD,DC.DAC.FINL.CD,DC.DAC.FRAL.CD,DC.DAC.GBRL.CD,DC.DAC.GRCL.CD,DC.DAC.IRLL.CD,DC.DAC.ISLL.CD,DC.DAC.ITAL.CD,DC.DAC.JPNL.CD,DC.DAC.KORL.CD,DC.DAC.LUXL.CD,DC.DAC.NLDL.CD,DC.DAC.NORL.CD,DC.DAC.NZLL.CD,DC.DAC.POLL.CD,DC.DAC.PRTL.CD,DC.DAC.SVKL.CD,DC.DAC.SVNL.CD,DC.DAC.SWEL.CD,DC.DAC.TOTL.CD,DC.DAC.USAL.CD,DC.ODA.TLDC.CD,DC.ODA.TLDC.GN.ZS,DC.ODA.TOTL.CD,DC.ODA.TOTL.GN.ZS,DC.ODA.TOTL.KD,DT.AMT.BLAT.CD,DT.AMT.BLTC.CD,DT.AMT.DIMF.CD,DT.AMT.DLTF.CD,DT.AMT.DLXF.CD,DT.AMT.DPNG.CD,DT.AMT.DPPG.CD,DT.AMT.MIBR.CD,DT.AMT.MIDA.CD,DT.AMT.MLAT.CD,DT.AMT.MLTC.CD,DT.AMT.OFFT.CD,DT.AMT.PBND.CD,DT.AMT.PCBK.CD,DT.AMT.PNGB.CD,DT.AMT.PNGC.CD,DT.AMT.PROP.CD,DT.AMT.PRVT.CD,DT.AXA.DPPG.CD,DT.AXA.OFFT.CD,DT.AXA.PRVT.CD,DT.AXF.DPPG.CD,DT.AXR.DPPG.CD,DT.AXR.OFFT.CD,DT.AXR.PRVT.CD,DT.COM.DPPG.CD,DT.COM.MIBR.CD,DT.COM.MIDA.CD,DT.COM.OFFT.CD,DT.COM.PRVT.CD,DT.CUR.DMAK.ZS,DT.CUR.EURO.ZS,DT.CUR.FFRC.ZS,DT.CUR.JYEN.ZS,DT.CUR.MULC.ZS,DT.CUR.OTHC.ZS,DT.CUR.SDRW.ZS,DT.CUR.SWFR.ZS,DT.CUR.UKPS.ZS,DT.CUR.USDL.ZS,DT.DFR.DPPG.CD,DT.DIS.BLAT.CD,DT.DIS.BLTC.CD,DT.DIS.DIMF.CD,DT.DIS.DLTF.CD,DT.DIS.DLXF.CD,DT.DIS.DPNG.CD,DT.DIS.DPPG.CD,DT.DIS.IDAG.CD,DT.DIS.MIBR.CD,DT.DIS.MIDA.CD,DT.DIS.MLAT.CD,DT.DIS.MLTC.CD,DT.DIS.OFFT.CD,DT.DIS.PBND.CD,DT.DIS.PCBK.CD,DT.DIS.PNGB.CD,DT.DIS.PNGC.CD,DT.DIS.PROP.CD,DT.DIS.PRVT.CD,DT.DOD.ALLC.CD,DT.DOD.ALLC.ZS,DT.DOD.BLAT.CD,DT.DOD.BLTC.CD,DT.DOD.DECT.CD,DT.DOD.DECT.CD.CG,DT.DOD.DECT.EX.ZS,DT.DOD.DECT.GN.ZS,DT.DOD.DIMF.CD,DT.DOD.DLXF.CD,DT.DOD.DPNG.CD,DT.DOD.DPPG.CD,DT.DOD.DSTC.CD,DT.DOD.DSTC.IR.ZS,DT.DOD.DSTC.XP.ZS,DT.DOD.DSTC.ZS,DT.DOD.MDRI.CD,DT.DOD.MIBR.CD,DT.DOD.MIDA.CD,DT.DOD.MLAT.CD,DT.DOD.MLAT.ZS,DT.DOD.MLTC.CD,DT.DOD.MWBG.CD,DT.DOD.OFFT.CD,DT.DOD.PBND.CD,DT.DOD.PCBK.CD,DT.DOD.PNGB.CD,DT.DOD.PNGC.CD,DT.DOD.PROP.CD,DT.DOD.PRVS.CD,DT.DOD.PRVT.CD,DT.DOD.PUBS.CD,DT.DOD.PVLX.CD,DT.DOD.PVLX.EX.ZS,DT.DOD.PVLX.GN.ZS,DT.DOD.RSDL.CD,DT.DOD.VTOT.CD,DT.DSB.DPPG.CD,DT.DSF.DPPG.CD,DT.DXR.DPPG.CD,DT.GPA.DPPG,DT.GPA.OFFT,DT.GPA.PRVT,DT.GRE.DPPG,DT.GRE.OFFT,DT.GRE.PRVT,DT.INR.DPPG,DT.INR.OFFT,DT.INR.PRVT,DT.INT.BLAT.CD,DT.INT.BLTC.CD,DT.INT.DECT.CD,DT.INT.DECT.EX.ZS,DT.INT.DECT.GN.ZS,DT.INT.DIMF.CD,DT.INT.DLXF.CD,DT.INT.DPNG.CD,DT.INT.DPPG.CD,DT.INT.DSTC.CD,DT.INT.MIBR.CD,DT.INT.MIDA.CD,DT.INT.MLAT.CD,DT.INT.MLTC.CD,DT.INT.OFFT.CD,DT.INT.PBND.CD,DT.INT.PCBK.CD,DT.INT.PNGB.CD,DT.INT.PNGC.CD,DT.INT.PROP.CD,DT.INT.PRVT.CD,DT.IXA.DPPG.CD,DT.IXA.DPPG.CD.CG,DT.IXA.OFFT.CD,DT.IXA.PRVT.CD,DT.IXF.DPPG.CD,DT.IXR.DPPG.CD,DT.IXR.OFFT.CD,DT.IXR.PRVT.CD,DT.MAT.DPPG,...,SL.UEM.1524.MA.ZS,SL.UEM.1524.NE.ZS,SL.UEM.1524.ZS,SL.UEM.ADVN.FE.ZS,SL.UEM.ADVN.MA.ZS,SL.UEM.ADVN.ZS,SL.UEM.BASC.FE.ZS,SL.UEM.BASC.MA.ZS,SL.UEM.BASC.ZS,SL.UEM.INTM.FE

In [55]:
eco_2016_sel.shape

(42, 1419)

### Delete columns with a lot of Null Values

In [56]:
eco_2016sel = eco_2016_sel.copy() #Cortafuegos por si el drop sale mal

In [57]:
drop = []
for column in eco_2016sel.columns:
    if eco_2016sel[column].isnull().sum() > 0:
        drop.append(column)
eco_2016sel = eco_2016sel.drop(drop, axis=1)

In [58]:
eco_2016sel.shape

(42, 374)

In [59]:
eco_2016sel.head()

Indicator Code,Country Code,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CROP.ZS,AG.LND.TOTL.K2,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EN.POP.DNST,EN.URB.LCTY,EN.URB.LCTY.UR.ZS,EP.PMP.SGAS.CD,ER.FSH.AQUA.MT,ER.FSH.CAPT.MT,ER.FSH.PROD.MT,ER.LND.PTLD.ZS,ER.PTD.TOTL.ZS,FI.RES.TOTL.CD,FI.RES.XGLD.CD,IC.BUS.DFRN.XQ,IC.BUS.DISC.XQ,IC.CRD.INFO.XQ,IC.CRD.PRVT.ZS,IC.CRD.PUBL.ZS,IC.ELC.TIME,IC.EXP.CSBC.CD,IC.EXP.CSDC.CD,IC.EXP.TMBC,IC.EXP.TMDC,IC.IMP.CSBC.CD,IC.IMP.CSDC.CD,IC.IMP.TMBC,IC.IMP.TMDC,IC.LGL.CRED.XQ,IC.LGL.DURS,IC.PRP.DURS,IC.PRP.PROC,IC.REG.COST.PC.FE.ZS,IC.REG.COST.PC.MA.ZS,IC.REG.COST.PC.ZS,IC.REG.DURS,IC.REG.DURS.FE,IC.REG.DURS.MA,IC.REG.PROC,IC.REG.PROC.FE,IC.REG.PROC.MA,IC.TAX.DURS,IC.TAX.LABR.CP.ZS,IC.TAX.OTHR.CP.ZS,IC.TAX.PAYM,IC.TAX.PRFT.CP.ZS,IC.TAX.TOTL.CP.ZS,IC.WRH.DURS,IC.WRH.PROC,IQ.WEF.CUST.XQ,IQ.WEF.PORT.XQ,IT.CEL.SETS,IT.CEL.SETS.P2,IT.MLT.MAIN,IT.MLT.MAIN.P2,IT.NET.BBND,IT.NET.BBND.P2,IT.NET.SECR,IT.NET.SECR.P6,IT.NET.USER.ZS,NE.CON.GOVT.CD,NE.CON.GOVT.CN,NE.CON.GOVT.ZS,NE.CON.PRVT.CD,NE.CON.PRVT.CN,NE.CON.PRVT.KD,NE.CON.PRVT.KD.ZG,NE.CON.PRVT.KN,NE.CON.PRVT.PC.KD,NE.CON.PRVT.PC.KD.ZG,NE.CON.PRVT.PP.KD,NE.CON.PRVT.ZS,NE.CON.TOTL.CD,NE.CON.TOTL.CN,NE.CON.TOTL.KD,NE.CON.TOTL.KD.ZG,NE.CON.TOTL.KN,NE.CON.TOTL.ZS,NE.DAB.DEFL.ZS,NE.DAB.TOTL.CD,NE.DAB.TOTL.CN,NE.DAB.TOTL.KD,NE.DAB.TOTL.KN,NE.DAB.TOTL.ZS,NE.EXP.GNFS.CD,NE.EXP.GNFS.CN,NE.EXP.GNFS.ZS,NE.GDI.FTOT.CD,NE.GDI.FTOT.CN,NE.GDI.FTOT.ZS,NE.GDI.TOTL.CD,NE.GDI.TOTL.CN,NE.GDI.TOTL.KD,NE.GDI.TOTL.KD.ZG,NE.GDI.TOTL.KN,NE.GDI.TOTL.ZS,NE.IMP.GNFS.CD,NE.IMP.GNFS.CN,NE.IMP.GNFS.ZS,NE.RSB.GNFS.CD,NE.RSB.GNFS.CN,NE.RSB.GNFS.ZS,NE.TRD.GNFS.ZS,NV.AGR.EMPL.KD,NV.AGR.TOTL.KD,NV.AGR.TOTL.KD.ZG,NV.AGR.TOTL.KN,NV.IND.EMPL.KD,NV.IND.TOTL.KD,NV.IND.TOTL.KD.ZG,NV.IND.TOTL.KN,NV.SRV.EMPL.KD,NV.SRV.TOTL.KD,NV.SRV.TOTL.KD.ZG,NV.SRV.TOTL.KN,NY.ADJ.DCO2.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DKAP.CD,NY.ADJ.DKAP.GN.ZS,NY.ADJ.DMIN.CD,NY.ADJ.DMIN.GN.ZS,NY.ADJ.DNGY.CD,NY.ADJ.DNGY.GN.ZS,NY.GDP.COAL.RT.ZS,NY.GDP.DEFL.KD.ZG,NY.GDP.DEFL.KD.ZG.AD,NY.GDP.DEFL.ZS,NY.GDP.DEFL.ZS.AD,NY.GDP.DISC.CN,NY.GDP.FRST.RT.ZS,NY.GDP.MINR.RT.ZS,NY.GDP.MKTP.CD,NY.GDP.MKTP.CN,NY.GDP.MKTP.CN.AD,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.MKTP.KN,NY.GDP.MKTP.PP.CD,NY.GDP.MKTP.PP.KD,NY.GDP.NGAS.RT.ZS,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,NY.GDP.PCAP.KN,NY.GDP.PCAP.PP.CD,NY.GDP.PCAP.PP.KD,NY.GDP.PETR.RT.ZS,NY.GDP.TOTL.RT.ZS,NY.GDS.TOTL.CD,NY.GDS.TOTL.CN,NY.GDS.TOTL.ZS,NY.GNP.ATLS.CD,NY.GNP.MKTP.CD,NY.GNP.MKTP.CN,NY.GNP.MKTP.KD,NY.GNP.MKTP.KD.ZG,NY.GNP.MKTP.KN,NY.GNP.MKTP.PP.CD,NY.GNP.MKTP.PP.KD,NY.GNP.PCAP.CD,NY.GNP.PCAP.CN,NY.GNP.PCAP.KD,NY.GNP.PCAP.KD.ZG,NY.GNP.PCAP.KN,NY.GNP.PCAP.PP.CD,NY.GNP.PCAP.PP.KD,NY.GSR.NFCY.CD,NY.GSR.NFCY.CN,PA.NUS.ATLS,PA.NUS.PPP,PA.NUS.PPPC.RF,SE.COM.DURS,SE.PRE.DURS,SE.PRM.AGES,SE.PRM.DURS,SE.PRM.ENRL,SE.PRM.ENRL.FE.ZS,SE.SEC.AGES,SE.SEC.DURS,SG.LAW.EQRM.WK,SG.LEG.DVAW,SH.TBS.DTEC.ZS,SH.TBS.INCD,SL.AGR.EMPL.FE.ZS,SL.AGR.EMPL.MA.ZS,SL.AGR.EMPL.ZS,SL.EMP.1524.SP.FE.ZS,SL.EMP.1524.SP.MA.ZS,SL.EMP.1524.SP.ZS,SL.EMP.MPYR.FE.ZS,SL.EMP.MPYR.MA.ZS,SL.EMP.MPYR.ZS,SL.EMP.SELF.FE.ZS,SL.EMP.SELF.MA.ZS,SL.EMP.SELF.ZS,SL.EMP.TOTL.SP.FE.ZS,SL.EMP.TOTL.SP.MA.ZS,SL.EMP.TOTL.SP.ZS,SL.EMP.VULN.FE.ZS,SL.EMP.VULN.MA.ZS,SL.EMP.VULN.ZS,SL.EMP.WORK.FE.ZS,SL.EMP.WORK.MA.ZS,SL.EMP.WORK.ZS,SL.FAM.WORK.FE.ZS,SL.FAM.WORK.MA.ZS,SL.FAM.WORK.ZS,SL.GDP.PCAP.EM.KD,SL.IND.EMPL.FE.ZS,SL.IND.EMPL.MA.ZS,SL.IND.EMPL.ZS,SL.SRV.EMPL.FE.ZS,SL.SRV.EMPL.MA.ZS,SL.SRV.EMPL.ZS,SL.TLF.ACTI.1524.FE.ZS,SL.TLF.ACTI.1524.MA.ZS,SL.TLF.ACTI.1524.ZS,SL.TLF.ACTI.FE.ZS,SL.TLF.ACTI.MA.ZS,SL.TLF.ACTI.ZS,SL.TLF.CACT.FE.ZS,SL.TLF.CACT.FM.ZS,SL.TLF.CACT.MA.ZS,SL.TLF.CACT.ZS,SL.TLF.TOTL.FE.ZS,SL.TLF.TOTL.IN,SL.UEM.1524.FE.ZS,SL.UEM.1524.MA.ZS,SL.UEM.1524.ZS,SL.UEM.TOTL.FE.ZS,SL.UEM.TOTL.MA.ZS,SL.UEM.TOTL.ZS,SM.POP.REFG.OR,SP.ADO.TFRT,SP.DYN.CBRT.IN,S

Nos quedan 374 indicadores para comparar con el Big Mac index.

### Merge Datasets

Big Mac index DataSet

In [60]:
bm_2016.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
0,2016-01-01,ARE,AED,UAE,13,3.6731,3.53924
1,2016-01-01,ARG,ARS,Argentina,33,13.8093,2.3897
2,2016-01-01,AUS,AUD,Australia,5.3,1.41573,3.74366
3,2016-01-01,BRA,BRL,Brazil,13.5,4.0248,3.3542
4,2016-01-01,CAN,CAD,Canada,5.84,1.4092,4.1442


Economic Data from countries dataset

In [61]:
eco_2016sel.head()

Indicator Code,Country Code,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CROP.ZS,AG.LND.TOTL.K2,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EN.POP.DNST,EN.URB.LCTY,EN.URB.LCTY.UR.ZS,EP.PMP.SGAS.CD,ER.FSH.AQUA.MT,ER.FSH.CAPT.MT,ER.FSH.PROD.MT,ER.LND.PTLD.ZS,ER.PTD.TOTL.ZS,FI.RES.TOTL.CD,FI.RES.XGLD.CD,IC.BUS.DFRN.XQ,IC.BUS.DISC.XQ,IC.CRD.INFO.XQ,IC.CRD.PRVT.ZS,IC.CRD.PUBL.ZS,IC.ELC.TIME,IC.EXP.CSBC.CD,IC.EXP.CSDC.CD,IC.EXP.TMBC,IC.EXP.TMDC,IC.IMP.CSBC.CD,IC.IMP.CSDC.CD,IC.IMP.TMBC,IC.IMP.TMDC,IC.LGL.CRED.XQ,IC.LGL.DURS,IC.PRP.DURS,IC.PRP.PROC,IC.REG.COST.PC.FE.ZS,IC.REG.COST.PC.MA.ZS,IC.REG.COST.PC.ZS,IC.REG.DURS,IC.REG.DURS.FE,IC.REG.DURS.MA,IC.REG.PROC,IC.REG.PROC.FE,IC.REG.PROC.MA,IC.TAX.DURS,IC.TAX.LABR.CP.ZS,IC.TAX.OTHR.CP.ZS,IC.TAX.PAYM,IC.TAX.PRFT.CP.ZS,IC.TAX.TOTL.CP.ZS,IC.WRH.DURS,IC.WRH.PROC,IQ.WEF.CUST.XQ,IQ.WEF.PORT.XQ,IT.CEL.SETS,IT.CEL.SETS.P2,IT.MLT.MAIN,IT.MLT.MAIN.P2,IT.NET.BBND,IT.NET.BBND.P2,IT.NET.SECR,IT.NET.SECR.P6,IT.NET.USER.ZS,NE.CON.GOVT.CD,NE.CON.GOVT.CN,NE.CON.GOVT.ZS,NE.CON.PRVT.CD,NE.CON.PRVT.CN,NE.CON.PRVT.KD,NE.CON.PRVT.KD.ZG,NE.CON.PRVT.KN,NE.CON.PRVT.PC.KD,NE.CON.PRVT.PC.KD.ZG,NE.CON.PRVT.PP.KD,NE.CON.PRVT.ZS,NE.CON.TOTL.CD,NE.CON.TOTL.CN,NE.CON.TOTL.KD,NE.CON.TOTL.KD.ZG,NE.CON.TOTL.KN,NE.CON.TOTL.ZS,NE.DAB.DEFL.ZS,NE.DAB.TOTL.CD,NE.DAB.TOTL.CN,NE.DAB.TOTL.KD,NE.DAB.TOTL.KN,NE.DAB.TOTL.ZS,NE.EXP.GNFS.CD,NE.EXP.GNFS.CN,NE.EXP.GNFS.ZS,NE.GDI.FTOT.CD,NE.GDI.FTOT.CN,NE.GDI.FTOT.ZS,NE.GDI.TOTL.CD,NE.GDI.TOTL.CN,NE.GDI.TOTL.KD,NE.GDI.TOTL.KD.ZG,NE.GDI.TOTL.KN,NE.GDI.TOTL.ZS,NE.IMP.GNFS.CD,NE.IMP.GNFS.CN,NE.IMP.GNFS.ZS,NE.RSB.GNFS.CD,NE.RSB.GNFS.CN,NE.RSB.GNFS.ZS,NE.TRD.GNFS.ZS,NV.AGR.EMPL.KD,NV.AGR.TOTL.KD,NV.AGR.TOTL.KD.ZG,NV.AGR.TOTL.KN,NV.IND.EMPL.KD,NV.IND.TOTL.KD,NV.IND.TOTL.KD.ZG,NV.IND.TOTL.KN,NV.SRV.EMPL.KD,NV.SRV.TOTL.KD,NV.SRV.TOTL.KD.ZG,NV.SRV.TOTL.KN,NY.ADJ.DCO2.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DKAP.CD,NY.ADJ.DKAP.GN.ZS,NY.ADJ.DMIN.CD,NY.ADJ.DMIN.GN.ZS,NY.ADJ.DNGY.CD,NY.ADJ.DNGY.GN.ZS,NY.GDP.COAL.RT.ZS,NY.GDP.DEFL.KD.ZG,NY.GDP.DEFL.KD.ZG.AD,NY.GDP.DEFL.ZS,NY.GDP.DEFL.ZS.AD,NY.GDP.DISC.CN,NY.GDP.FRST.RT.ZS,NY.GDP.MINR.RT.ZS,NY.GDP.MKTP.CD,NY.GDP.MKTP.CN,NY.GDP.MKTP.CN.AD,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.MKTP.KN,NY.GDP.MKTP.PP.CD,NY.GDP.MKTP.PP.KD,NY.GDP.NGAS.RT.ZS,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,NY.GDP.PCAP.KN,NY.GDP.PCAP.PP.CD,NY.GDP.PCAP.PP.KD,NY.GDP.PETR.RT.ZS,NY.GDP.TOTL.RT.ZS,NY.GDS.TOTL.CD,NY.GDS.TOTL.CN,NY.GDS.TOTL.ZS,NY.GNP.ATLS.CD,NY.GNP.MKTP.CD,NY.GNP.MKTP.CN,NY.GNP.MKTP.KD,NY.GNP.MKTP.KD.ZG,NY.GNP.MKTP.KN,NY.GNP.MKTP.PP.CD,NY.GNP.MKTP.PP.KD,NY.GNP.PCAP.CD,NY.GNP.PCAP.CN,NY.GNP.PCAP.KD,NY.GNP.PCAP.KD.ZG,NY.GNP.PCAP.KN,NY.GNP.PCAP.PP.CD,NY.GNP.PCAP.PP.KD,NY.GSR.NFCY.CD,NY.GSR.NFCY.CN,PA.NUS.ATLS,PA.NUS.PPP,PA.NUS.PPPC.RF,SE.COM.DURS,SE.PRE.DURS,SE.PRM.AGES,SE.PRM.DURS,SE.PRM.ENRL,SE.PRM.ENRL.FE.ZS,SE.SEC.AGES,SE.SEC.DURS,SG.LAW.EQRM.WK,SG.LEG.DVAW,SH.TBS.DTEC.ZS,SH.TBS.INCD,SL.AGR.EMPL.FE.ZS,SL.AGR.EMPL.MA.ZS,SL.AGR.EMPL.ZS,SL.EMP.1524.SP.FE.ZS,SL.EMP.1524.SP.MA.ZS,SL.EMP.1524.SP.ZS,SL.EMP.MPYR.FE.ZS,SL.EMP.MPYR.MA.ZS,SL.EMP.MPYR.ZS,SL.EMP.SELF.FE.ZS,SL.EMP.SELF.MA.ZS,SL.EMP.SELF.ZS,SL.EMP.TOTL.SP.FE.ZS,SL.EMP.TOTL.SP.MA.ZS,SL.EMP.TOTL.SP.ZS,SL.EMP.VULN.FE.ZS,SL.EMP.VULN.MA.ZS,SL.EMP.VULN.ZS,SL.EMP.WORK.FE.ZS,SL.EMP.WORK.MA.ZS,SL.EMP.WORK.ZS,SL.FAM.WORK.FE.ZS,SL.FAM.WORK.MA.ZS,SL.FAM.WORK.ZS,SL.GDP.PCAP.EM.KD,SL.IND.EMPL.FE.ZS,SL.IND.EMPL.MA.ZS,SL.IND.EMPL.ZS,SL.SRV.EMPL.FE.ZS,SL.SRV.EMPL.MA.ZS,SL.SRV.EMPL.ZS,SL.TLF.ACTI.1524.FE.ZS,SL.TLF.ACTI.1524.MA.ZS,SL.TLF.ACTI.1524.ZS,SL.TLF.ACTI.FE.ZS,SL.TLF.ACTI.MA.ZS,SL.TLF.ACTI.ZS,SL.TLF.CACT.FE.ZS,SL.TLF.CACT.FM.ZS,SL.TLF.CACT.MA.ZS,SL.TLF.CACT.ZS,SL.TLF.TOTL.FE.ZS,SL.TLF.TOTL.IN,SL.UEM.1524.FE.ZS,SL.UEM.1524.MA.ZS,SL.UEM.1524.ZS,SL.UEM.TOTL.FE.ZS,SL.UEM.TOTL.MA.ZS,SL.UEM.TOTL.ZS,SM.POP.REFG.OR,SP.ADO.TFRT,SP.DYN.CBRT.IN,S

In [62]:
eco_2016sel.rename(columns={'Country Code':'iso_a3'}, inplace=True)

In [63]:
eco_2016sel.head()

Indicator Code,iso_a3,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CROP.ZS,AG.LND.TOTL.K2,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EN.POP.DNST,EN.URB.LCTY,EN.URB.LCTY.UR.ZS,EP.PMP.SGAS.CD,ER.FSH.AQUA.MT,ER.FSH.CAPT.MT,ER.FSH.PROD.MT,ER.LND.PTLD.ZS,ER.PTD.TOTL.ZS,FI.RES.TOTL.CD,FI.RES.XGLD.CD,IC.BUS.DFRN.XQ,IC.BUS.DISC.XQ,IC.CRD.INFO.XQ,IC.CRD.PRVT.ZS,IC.CRD.PUBL.ZS,IC.ELC.TIME,IC.EXP.CSBC.CD,IC.EXP.CSDC.CD,IC.EXP.TMBC,IC.EXP.TMDC,IC.IMP.CSBC.CD,IC.IMP.CSDC.CD,IC.IMP.TMBC,IC.IMP.TMDC,IC.LGL.CRED.XQ,IC.LGL.DURS,IC.PRP.DURS,IC.PRP.PROC,IC.REG.COST.PC.FE.ZS,IC.REG.COST.PC.MA.ZS,IC.REG.COST.PC.ZS,IC.REG.DURS,IC.REG.DURS.FE,IC.REG.DURS.MA,IC.REG.PROC,IC.REG.PROC.FE,IC.REG.PROC.MA,IC.TAX.DURS,IC.TAX.LABR.CP.ZS,IC.TAX.OTHR.CP.ZS,IC.TAX.PAYM,IC.TAX.PRFT.CP.ZS,IC.TAX.TOTL.CP.ZS,IC.WRH.DURS,IC.WRH.PROC,IQ.WEF.CUST.XQ,IQ.WEF.PORT.XQ,IT.CEL.SETS,IT.CEL.SETS.P2,IT.MLT.MAIN,IT.MLT.MAIN.P2,IT.NET.BBND,IT.NET.BBND.P2,IT.NET.SECR,IT.NET.SECR.P6,IT.NET.USER.ZS,NE.CON.GOVT.CD,NE.CON.GOVT.CN,NE.CON.GOVT.ZS,NE.CON.PRVT.CD,NE.CON.PRVT.CN,NE.CON.PRVT.KD,NE.CON.PRVT.KD.ZG,NE.CON.PRVT.KN,NE.CON.PRVT.PC.KD,NE.CON.PRVT.PC.KD.ZG,NE.CON.PRVT.PP.KD,NE.CON.PRVT.ZS,NE.CON.TOTL.CD,NE.CON.TOTL.CN,NE.CON.TOTL.KD,NE.CON.TOTL.KD.ZG,NE.CON.TOTL.KN,NE.CON.TOTL.ZS,NE.DAB.DEFL.ZS,NE.DAB.TOTL.CD,NE.DAB.TOTL.CN,NE.DAB.TOTL.KD,NE.DAB.TOTL.KN,NE.DAB.TOTL.ZS,NE.EXP.GNFS.CD,NE.EXP.GNFS.CN,NE.EXP.GNFS.ZS,NE.GDI.FTOT.CD,NE.GDI.FTOT.CN,NE.GDI.FTOT.ZS,NE.GDI.TOTL.CD,NE.GDI.TOTL.CN,NE.GDI.TOTL.KD,NE.GDI.TOTL.KD.ZG,NE.GDI.TOTL.KN,NE.GDI.TOTL.ZS,NE.IMP.GNFS.CD,NE.IMP.GNFS.CN,NE.IMP.GNFS.ZS,NE.RSB.GNFS.CD,NE.RSB.GNFS.CN,NE.RSB.GNFS.ZS,NE.TRD.GNFS.ZS,NV.AGR.EMPL.KD,NV.AGR.TOTL.KD,NV.AGR.TOTL.KD.ZG,NV.AGR.TOTL.KN,NV.IND.EMPL.KD,NV.IND.TOTL.KD,NV.IND.TOTL.KD.ZG,NV.IND.TOTL.KN,NV.SRV.EMPL.KD,NV.SRV.TOTL.KD,NV.SRV.TOTL.KD.ZG,NV.SRV.TOTL.KN,NY.ADJ.DCO2.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DKAP.CD,NY.ADJ.DKAP.GN.ZS,NY.ADJ.DMIN.CD,NY.ADJ.DMIN.GN.ZS,NY.ADJ.DNGY.CD,NY.ADJ.DNGY.GN.ZS,NY.GDP.COAL.RT.ZS,NY.GDP.DEFL.KD.ZG,NY.GDP.DEFL.KD.ZG.AD,NY.GDP.DEFL.ZS,NY.GDP.DEFL.ZS.AD,NY.GDP.DISC.CN,NY.GDP.FRST.RT.ZS,NY.GDP.MINR.RT.ZS,NY.GDP.MKTP.CD,NY.GDP.MKTP.CN,NY.GDP.MKTP.CN.AD,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.MKTP.KN,NY.GDP.MKTP.PP.CD,NY.GDP.MKTP.PP.KD,NY.GDP.NGAS.RT.ZS,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,NY.GDP.PCAP.KN,NY.GDP.PCAP.PP.CD,NY.GDP.PCAP.PP.KD,NY.GDP.PETR.RT.ZS,NY.GDP.TOTL.RT.ZS,NY.GDS.TOTL.CD,NY.GDS.TOTL.CN,NY.GDS.TOTL.ZS,NY.GNP.ATLS.CD,NY.GNP.MKTP.CD,NY.GNP.MKTP.CN,NY.GNP.MKTP.KD,NY.GNP.MKTP.KD.ZG,NY.GNP.MKTP.KN,NY.GNP.MKTP.PP.CD,NY.GNP.MKTP.PP.KD,NY.GNP.PCAP.CD,NY.GNP.PCAP.CN,NY.GNP.PCAP.KD,NY.GNP.PCAP.KD.ZG,NY.GNP.PCAP.KN,NY.GNP.PCAP.PP.CD,NY.GNP.PCAP.PP.KD,NY.GSR.NFCY.CD,NY.GSR.NFCY.CN,PA.NUS.ATLS,PA.NUS.PPP,PA.NUS.PPPC.RF,SE.COM.DURS,SE.PRE.DURS,SE.PRM.AGES,SE.PRM.DURS,SE.PRM.ENRL,SE.PRM.ENRL.FE.ZS,SE.SEC.AGES,SE.SEC.DURS,SG.LAW.EQRM.WK,SG.LEG.DVAW,SH.TBS.DTEC.ZS,SH.TBS.INCD,SL.AGR.EMPL.FE.ZS,SL.AGR.EMPL.MA.ZS,SL.AGR.EMPL.ZS,SL.EMP.1524.SP.FE.ZS,SL.EMP.1524.SP.MA.ZS,SL.EMP.1524.SP.ZS,SL.EMP.MPYR.FE.ZS,SL.EMP.MPYR.MA.ZS,SL.EMP.MPYR.ZS,SL.EMP.SELF.FE.ZS,SL.EMP.SELF.MA.ZS,SL.EMP.SELF.ZS,SL.EMP.TOTL.SP.FE.ZS,SL.EMP.TOTL.SP.MA.ZS,SL.EMP.TOTL.SP.ZS,SL.EMP.VULN.FE.ZS,SL.EMP.VULN.MA.ZS,SL.EMP.VULN.ZS,SL.EMP.WORK.FE.ZS,SL.EMP.WORK.MA.ZS,SL.EMP.WORK.ZS,SL.FAM.WORK.FE.ZS,SL.FAM.WORK.MA.ZS,SL.FAM.WORK.ZS,SL.GDP.PCAP.EM.KD,SL.IND.EMPL.FE.ZS,SL.IND.EMPL.MA.ZS,SL.IND.EMPL.ZS,SL.SRV.EMPL.FE.ZS,SL.SRV.EMPL.MA.ZS,SL.SRV.EMPL.ZS,SL.TLF.ACTI.1524.FE.ZS,SL.TLF.ACTI.1524.MA.ZS,SL.TLF.ACTI.1524.ZS,SL.TLF.ACTI.FE.ZS,SL.TLF.ACTI.MA.ZS,SL.TLF.ACTI.ZS,SL.TLF.CACT.FE.ZS,SL.TLF.CACT.FM.ZS,SL.TLF.CACT.MA.ZS,SL.TLF.CACT.ZS,SL.TLF.TOTL.FE.ZS,SL.TLF.TOTL.IN,SL.UEM.1524.FE.ZS,SL.UEM.1524.MA.ZS,SL.UEM.1524.ZS,SL.UEM.TOTL.FE.ZS,SL.UEM.TOTL.MA.ZS,SL.UEM.TOTL.ZS,SM.POP.REFG.OR,SP.ADO.TFRT,SP.DYN.CBRT.IN,SP.DYN.

In [64]:
data = pd.merge(bm_2016, eco_2016sel, how ='inner', on ='iso_a3')

In [65]:
#Some of the values in collar_price are not float type
data['dollar_price'] = data['dollar_price'].apply(lambda x: float(x))

In [66]:
data['region'] = ['MidleEast','LatinAmerica','Oceania','LatinAmerica',
 'NortAmerica','Europe','LatinAmerica','Asia','LatinAmerica',
 'LatinAmerica','Europe','Europe','Africa','Europe','Asia',
 'Europe','Asia','Asia','MidleEast','Asia','Asia','Asia',
 'LatinAmerica','Asia','Europe','Oceania','MidleEast',
 'LatinAmerica','Asia','Europe','Europe','MidleEast',
 'Asia','Europe','Asia','MidleEast','Europe','LatinAmerica',
 'NortAmerica','Asia','Africa','Europe']

In [67]:
data['continent'] = ['Asia','America','Oceania','America',
 'America','Europe','America','Asia','America',
 'America','Europe','Europe','Africa','Europe','Asia',
 'Europe','Asia','Asia','Asia','Asia','Asia','Asia',
 'America','Asia','Europe','Oceania','Asia',
 'America','Asia','Europe','Europe','Asia',
 'Asia','Europe','Asia','Asia','Europe','America',
 'America','Asia','Africa','Europe']

In [68]:
data.head()

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,AG.LND.CROP.ZS,AG.LND.TOTL.K2,AG.PRD.CROP.XD,AG.PRD.FOOD.XD,AG.PRD.LVSK.XD,AG.SRF.TOTL.K2,BM.KLT.DINV.CD.WD,BM.KLT.DINV.WD.GD.ZS,BX.KLT.DINV.CD.WD,BX.KLT.DINV.WD.GD.ZS,EG.ELC.ACCS.RU.ZS,EG.ELC.ACCS.UR.ZS,EG.ELC.ACCS.ZS,EN.POP.DNST,EN.URB.LCTY,EN.URB.LCTY.UR.ZS,EP.PMP.SGAS.CD,ER.FSH.AQUA.MT,ER.FSH.CAPT.MT,ER.FSH.PROD.MT,ER.LND.PTLD.ZS,ER.PTD.TOTL.ZS,FI.RES.TOTL.CD,FI.RES.XGLD.CD,IC.BUS.DFRN.XQ,IC.BUS.DISC.XQ,IC.CRD.INFO.XQ,IC.CRD.PRVT.ZS,IC.CRD.PUBL.ZS,IC.ELC.TIME,IC.EXP.CSBC.CD,IC.EXP.CSDC.CD,IC.EXP.TMBC,IC.EXP.TMDC,IC.IMP.CSBC.CD,IC.IMP.CSDC.CD,IC.IMP.TMBC,IC.IMP.TMDC,IC.LGL.CRED.XQ,IC.LGL.DURS,IC.PRP.DURS,IC.PRP.PROC,IC.REG.COST.PC.FE.ZS,IC.REG.COST.PC.MA.ZS,IC.REG.COST.PC.ZS,IC.REG.DURS,IC.REG.DURS.FE,IC.REG.DURS.MA,IC.REG.PROC,IC.REG.PROC.FE,IC.REG.PROC.MA,IC.TAX.DURS,IC.TAX.LABR.CP.ZS,IC.TAX.OTHR.CP.ZS,IC.TAX.PAYM,IC.TAX.PRFT.CP.ZS,IC.TAX.TOTL.CP.ZS,IC.WRH.DURS,IC.WRH.PROC,IQ.WEF.CUST.XQ,IQ.WEF.PORT.XQ,IT.CEL.SETS,IT.CEL.SETS.P2,IT.MLT.MAIN,IT.MLT.MAIN.P2,IT.NET.BBND,IT.NET.BBND.P2,IT.NET.SECR,IT.NET.SECR.P6,IT.NET.USER.ZS,NE.CON.GOVT.CD,NE.CON.GOVT.CN,NE.CON.GOVT.ZS,NE.CON.PRVT.CD,NE.CON.PRVT.CN,NE.CON.PRVT.KD,NE.CON.PRVT.KD.ZG,NE.CON.PRVT.KN,NE.CON.PRVT.PC.KD,NE.CON.PRVT.PC.KD.ZG,NE.CON.PRVT.PP.KD,NE.CON.PRVT.ZS,NE.CON.TOTL.CD,NE.CON.TOTL.CN,NE.CON.TOTL.KD,NE.CON.TOTL.KD.ZG,NE.CON.TOTL.KN,NE.CON.TOTL.ZS,NE.DAB.DEFL.ZS,NE.DAB.TOTL.CD,NE.DAB.TOTL.CN,NE.DAB.TOTL.KD,NE.DAB.TOTL.KN,NE.DAB.TOTL.ZS,NE.EXP.GNFS.CD,NE.EXP.GNFS.CN,NE.EXP.GNFS.ZS,NE.GDI.FTOT.CD,NE.GDI.FTOT.CN,NE.GDI.FTOT.ZS,NE.GDI.TOTL.CD,NE.GDI.TOTL.CN,NE.GDI.TOTL.KD,NE.GDI.TOTL.KD.ZG,NE.GDI.TOTL.KN,NE.GDI.TOTL.ZS,NE.IMP.GNFS.CD,NE.IMP.GNFS.CN,NE.IMP.GNFS.ZS,NE.RSB.GNFS.CD,NE.RSB.GNFS.CN,NE.RSB.GNFS.ZS,NE.TRD.GNFS.ZS,NV.AGR.EMPL.KD,NV.AGR.TOTL.KD,NV.AGR.TOTL.KD.ZG,NV.AGR.TOTL.KN,NV.IND.EMPL.KD,NV.IND.TOTL.KD,NV.IND.TOTL.KD.ZG,NV.IND.TOTL.KN,NV.SRV.EMPL.KD,NV.SRV.TOTL.KD,NV.SRV.TOTL.KD.ZG,NV.SRV.TOTL.KN,NY.ADJ.DCO2.CD,NY.ADJ.DCO2.GN.ZS,NY.ADJ.DKAP.CD,NY.ADJ.DKAP.GN.ZS,NY.ADJ.DMIN.CD,NY.ADJ.DMIN.GN.ZS,NY.ADJ.DNGY.CD,NY.ADJ.DNGY.GN.ZS,NY.GDP.COAL.RT.ZS,NY.GDP.DEFL.KD.ZG,NY.GDP.DEFL.KD.ZG.AD,NY.GDP.DEFL.ZS,NY.GDP.DEFL.ZS.AD,NY.GDP.DISC.CN,NY.GDP.FRST.RT.ZS,NY.GDP.MINR.RT.ZS,NY.GDP.MKTP.CD,NY.GDP.MKTP.CN,NY.GDP.MKTP.CN.AD,NY.GDP.MKTP.KD,NY.GDP.MKTP.KD.ZG,NY.GDP.MKTP.KN,NY.GDP.MKTP.PP.CD,NY.GDP.MKTP.PP.KD,NY.GDP.NGAS.RT.ZS,NY.GDP.PCAP.CD,NY.GDP.PCAP.CN,NY.GDP.PCAP.KD,NY.GDP.PCAP.KD.ZG,NY.GDP.PCAP.KN,NY.GDP.PCAP.PP.CD,NY.GDP.PCAP.PP.KD,NY.GDP.PETR.RT.ZS,NY.GDP.TOTL.RT.ZS,NY.GDS.TOTL.CD,NY.GDS.TOTL.CN,NY.GDS.TOTL.ZS,NY.GNP.ATLS.CD,NY.GNP.MKTP.CD,NY.GNP.MKTP.CN,NY.GNP.MKTP.KD,NY.GNP.MKTP.KD.ZG,NY.GNP.MKTP.KN,NY.GNP.MKTP.PP.CD,NY.GNP.MKTP.PP.KD,NY.GNP.PCAP.CD,NY.GNP.PCAP.CN,NY.GNP.PCAP.KD,NY.GNP.PCAP.KD.ZG,NY.GNP.PCAP.KN,NY.GNP.PCAP.PP.CD,NY.GNP.PCAP.PP.KD,NY.GSR.NFCY.CD,NY.GSR.NFCY.CN,PA.NUS.ATLS,PA.NUS.PPP,PA.NUS.PPPC.RF,SE.COM.DURS,SE.PRE.DURS,SE.PRM.AGES,SE.PRM.DURS,SE.PRM.ENRL,SE.PRM.ENRL.FE.ZS,SE.SEC.AGES,SE.SEC.DURS,SG.LAW.EQRM.WK,SG.LEG.DVAW,SH.TBS.DTEC.ZS,SH.TBS.INCD,SL.AGR.EMPL.FE.ZS,SL.AGR.EMPL.MA.ZS,SL.AGR.EMPL.ZS,SL.EMP.1524.SP.FE.ZS,SL.EMP.1524.SP.MA.ZS,SL.EMP.1524.SP.ZS,SL.EMP.MPYR.FE.ZS,SL.EMP.MPYR.MA.ZS,SL.EMP.MPYR.ZS,SL.EMP.SELF.FE.ZS,SL.EMP.SELF.MA.ZS,SL.EMP.SELF.ZS,SL.EMP.TOTL.SP.FE.ZS,SL.EMP.TOTL.SP.MA.ZS,SL.EMP.TOTL.SP.ZS,SL.EMP.VULN.FE.ZS,SL.EMP.VULN.MA.ZS,SL.EMP.VULN.ZS,SL.EMP.WORK.FE.ZS,SL.EMP.WORK.MA.ZS,SL.EMP.WORK.ZS,SL.FAM.WORK.FE.ZS,SL.FAM.WORK.MA.ZS,SL.FAM.WORK.ZS,SL.GDP.PCAP.EM.KD,SL.IND.EMPL.FE.ZS,SL.IND.EMPL.MA.ZS,SL.IND.EMPL.ZS,SL.SRV.EMPL.FE.ZS,SL.SRV.EMPL.MA.ZS,SL.SRV.EMPL.ZS,SL.TLF.ACTI.1524.FE.ZS,SL.TLF.ACTI.1524.MA.ZS,SL.TLF.ACTI.1524.ZS,SL.TLF.ACTI.FE.ZS,SL.TLF.ACTI.MA.ZS,SL.TLF.ACTI.ZS,SL.TLF.CACT.FE.ZS,SL.TLF.CACT.FM.ZS,SL.TLF.CACT.MA.ZS,SL.TLF.CACT.ZS,SL.TLF.TOTL.FE.ZS,SL.TLF.TOTL.IN,SL.UEM.1524.FE.ZS,SL.UEM.1524.MA.ZS,SL.UEM.1524.ZS,SL.UEM.TOTL.FE.ZS,SL.UEM.TOTL.MA.ZS,SL.UEM.TOTL.ZS,SM.P

In [69]:
data.shape

(42, 382)

Save the resultant dataset into data

In [70]:
data.to_csv("data/clean_data.csv")

In [71]:
data_indicators = pd.DataFrame()

In [72]:
data_indicators['Indicator Code'] = data.columns[7:-1]

In [73]:
data_indicators = pd.merge(data_indicators, indicators, how ='inner', on ='Indicator Code')

In [74]:
data_indicators.head()

,Indicator Code,Indicator Name
0,AG.LND.AGRI.K2,Agricultural land (sq. km)
1,AG.LND.AGRI.ZS,Agricultural land (% of land area)
2,AG.LND.ARBL.HA,Arable land (hectares)
3,AG.LND.ARBL.HA.PC,Arable land (hectares per person)
4,AG.LND.ARBL.ZS,Arable land (% of land area)


**List of the indicators of the Dataset**

In [75]:
#list the indicators contained on the final data 
#Wich explain the rowns on the final dataset
for indicator in data_indicators['Indicator Name']:
    print(indicator)

Agricultural land (sq. km)
Agricultural land (% of land area)
Arable land (hectares)
Arable land (hectares per person)
Arable land (% of land area)
Permanent cropland (% of land area)
Land area (sq. km)
Crop production index (2004-2006 = 100)
Food production index (2004-2006 = 100)
Livestock production index (2004-2006 = 100)
Surface area (sq. km)
Foreign direct investment, net outflows (BoP, current US$)
Foreign direct investment, net outflows (% of GDP)
Foreign direct investment, net inflows (BoP, current US$)
Foreign direct investment, net inflows (% of GDP)
Access to electricity, rural (% of rural population)
Access to electricity, urban (% of urban population)
Access to electricity (% of population)
Population density (people per sq. km of land area)
Population in largest city
Population in the largest city (% of urban population)
Pump price for gasoline (US$ per liter)
Aquaculture production (metric tons)
Capture fisheries production (metric tons)
Total fisheries production (metr

We also save the indicators dataset.

In [77]:
data_indicators.to_csv("data/clean_data_indicators.csv")

Now we will continue the Statistical Analysis in another jupyter notebook.